---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    data = []
    state = None
    state_towns = []
    with open('university_towns.txt') as file:
        for line in file:
            editline = line[:-1]
            if editline[-6:] == '[edit]':
                state = editline[:-6]
                continue
            if '(' in line:
                town = editline[:editline.index('(')-1]
                state_towns.append([state,town])
            else:
                town = editline
                state_towns.append([state,town])
            data.append(editline)
    df = pd.DataFrame(state_towns,columns = ['State','RegionName'])
    return df
get_list_of_university_towns() 

,State,RegionName
0,None,
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
9,Alaska,Fairbanks


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.ExcelFile('gdplev.xls')
    GDP = GDP.parse("Sheet1", skiprows=219)
    GDP = GDP[['1999q4', 9926.1]]
    GDP.columns = ['Quarter','GDP']
    for i in range(2, len(GDP)):
        if (GDP.iloc[i-2][1] > GDP.iloc[i-1][1]) and (GDP.iloc[i-1][1] > GDP.iloc[i][1]):
            return GDP.iloc[i-2][0]
get_recession_start()

u'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP = pd.ExcelFile('gdplev.xls')
    GDP = GDP.parse("Sheet1", skiprows=219)
    GDP = GDP[['1999q4', 9926.1]]
    GDP.columns = ['Quarter','GDP']
    start = get_recession_start()
    start_index = GDP[GDP['Quarter'] == start].index.tolist()[0]
    GDP=GDP.iloc[start_index:]
    for i in range(2, len(GDP)):
        if (GDP.iloc[i-2][1] < GDP.iloc[i-1][1]) and (GDP.iloc[i-1][1] < GDP.iloc[i][1]):
            return GDP.iloc[i][0]
get_recession_end()

u'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = pd.ExcelFile('gdplev.xls')
    GDP = GDP.parse("Sheet1", skiprows=219)
    GDP = GDP[['1999q4', 9926.1]]
    GDP.columns = ['Quarter','GDP']
    start = get_recession_start()
    start_index = GDP[GDP['Quarter'] == start].index.tolist()[0]
    end = get_recession_end()
    end_index = GDP[GDP['Quarter'] == end].index.tolist()[0]
    GDP=GDP.iloc[start_index:end_index+1]
    bottom = GDP['GDP'].min()
    bottom_index = GDP[GDP['GDP'] == bottom].index.tolist()[0]-start_index
    return GDP.iloc[bottom_index]['Quarter']
get_recession_bottom()

u'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    houses = pd.read_csv('City_Zhvi_AllHomes.csv')
    for i in range(2000,2017):
        if i == 2016:
            houses[str(i) + 'q1'] = houses[[str(i)+'-01', str(i)+'-02', str(i)+'-03']].mean(axis=1)
            houses[str(i) + 'q2'] = houses[[str(i)+'-04', str(i)+'-05', str(i)+'-06']].mean(axis=1)
            houses[str(i) + 'q3'] = houses[[str(i)+'-07', str(i)+'-08']].mean(axis=1)
        else:
            houses[str(i) + 'q1'] = houses[[str(i)+'-01', str(i)+'-02', str(i)+'-03']].mean(axis=1)
            houses[str(i) + 'q2'] = houses[[str(i)+'-04', str(i)+'-05', str(i)+'-06']].mean(axis=1)
            houses[str(i) + 'q3'] = houses[[str(i)+'-07', str(i)+'-08', str(i)+'-09']].mean(axis=1)
            houses[str(i) + 'q4'] = houses[[str(i)+'-10', str(i)+'-11', str(i)+'-12']].mean(axis=1)

    houses = houses.drop(houses.columns[[0] + list(range(3,251))],axis=1)
    houses = houses.replace({'State':states})
    houses = houses.set_index(['State', 'RegionName'])
    return houses

convert_housing_data_to_quarters()



,,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2000q1,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,609600,616600,623300,628200.0,632400,639700,649400,661000,672400,NaN,...,5.153333e+05,5.191000e+05,5.221667e+05,5.235000e+05,5.320333e+05,5.485000e+05,5.619000e+05,5.721333e+05,5.841667e+05,599700.0
California,Los Angeles,593400,598700,605300,610200.0,613300,617600,623000,627200,629900,2.070667e+05,...,4.984000e+05,5.091333e+05,5.178667e+05,5.266667e+05,5.351333e+05,5.453000e+05,5.570333e+05,5.679000e+05,5.809000e+05,589150.0
Illinois,Chicago,209100,210900,212800,215400.0,218400,220600,221400,222000,222700,1.384000e+05,...,1.881333e+05,1.902667e+05,1.937333e+05,1.948667e+05,1.988667e+05,2.015667e+05,2.029000e+05,2.037667e+05,2.057667e+05,207550.0
Pennsylvania,Philadelphia,131600,132000,132400,133400.0,134600,135700,136300,136800,137300,5.300000e+04,...,1.146333e+05,1.158667e+05,1.166000e+05,1.167000e+05,1.179000e+05,1.206333e+05,1.221667e+05,1.237667e+05,1.260667e+05,130000.0
Arizona,Phoenix,199700,201100,203100,205000.0,206500,207800,209000,210000,211300,1.118333e+05,...,1.663333e+05,1.675333e+05,1.704667e+05,1.732667e+05,1.765000e+05,1.805667e+05,1.851667e+05,1.889000e+05,1.931667e+05,198150.0
Nevada,Las Vegas,204100,205400,206700,208200.0,209800,211300,212700,214500,216500,1.326000e+05,...,1.689667e+05,1.715667e+05,1.746000e+05,1.775667e+05,1.802667e+05,1.851667e+05,1.887333e+05,1.922333e+05,1.961333e+05,202150.0
California,San Diego,546600,548400,551500,554500.0,556000,557300,560000,565600,572100,2.229000e+05,...,4.747667e+05,4.814000e+05,4.861667e+05,4.934333e+05,5.034000e+05,5.130333e+05,5.209000e+05,5.273000e+05,5.321000e+05,541550.0
Texas,Dallas,157200,158400,159600,160500.0,161500,162600,163500,164200,164700,8.446667e+04,...,1.052667e+05,1.072667e+05,1.099667e+05,1.124000e+05,1.199667e+05,1.281667e+05,1.345667e+05,1.397000e+05,1.452333e+05,154750.0
California,San Jose,837600,841600,844100,848500.0,854700,860500,865900,872100,877400,3.742667e+05,...,6.817000e+05,7.016000e+05,7.184667e+05,7.371667e+05,7.637000e+05,7.885000e+05,8.060333e+05,8.242333e+05,8.384333e+05,835000.0


In [9]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    data = convert_housing_data_to_quarters().copy()
    print data
    data = data.loc[:,'2008q3':'2009q2']
    data = data.reset_index()
    def price_ratio(row):
        return (row['2008q3'] - row['2009q2'])/row['2008q3']
    
    data['up&down'] = data.apply(price_ratio,axis=1)
    
    
    uni_town = get_list_of_university_towns()['RegionName']
    uni_town = set(uni_town)

    def is_uni_town(row):
        if row['RegionName'] in uni_town:
            return 1
        else:
            return 0
    data['is_uni'] = data.apply(is_uni_town,axis=1)
    
    
    not_uni = data[data['is_uni']==0].loc[:,'up&down'].dropna()
    is_uni  = data[data['is_uni']==1].loc[:,'up&down'].dropna()
    def better():
        if not_uni.mean() < is_uni.mean():
            return 'non-university town'
        else:
            return 'university town'
    p_val = list(ttest_ind(not_uni, is_uni))[1]
    result = (True,p_val,better())
    return result


run_ttest()

                                          2016-09  2016-10  2016-11  \
State                RegionName                                       
New York             New York              609600   616600   623300   
California           Los Angeles           593400   598700   605300   
Illinois             Chicago               209100   210900   212800   
Pennsylvania         Philadelphia          131600   132000   132400   
Arizona              Phoenix               199700   201100   203100   
Nevada               Las Vegas             204100   205400   206700   
California           San Diego             546600   548400   551500   
Texas                Dallas                157200   158400   159600   
California           San Jose              837600   841600   844100   
Florida              Jacksonville          141100   143000   144900   
California           San Francisco        1120100  1121500  1133800   
Texas                Austin                302400   304100   305800   
Michig

(True, 0.00068653260304780647, 'university town')